# Assignment 3

In this assignment, we will focus on healthcare. This data set is made available by MIT. It contains data about 9,026 heartbeat measurements. Each row represents a single measurement (captured on a timeline). There are a total of 80 data points (columns). This is a multiclass classification task: predict whether the measurement represents a normal heartbeat or other anomalies. 

## Description of Variables

You will use the **hearbeat_cleaned.csv** data set for this assignment. Each row represents a single measurement. Columns labeled as T1 from T80 are the time steps on the timeline (there are 80 time steps, each time step has only one measurement). 

The last column is the target variable. It shows the label (category) of the measurement as follows:<br>
0 = Normal<br>
1 = Supraventricular premature beat<br>
2 = Premature ventricular contraction<br>
3 = Fusion of ventricular and normal beat<br>
4 = Unclassifiable beat

## Goal

Use the data set **hearbeat_cleaned.csv** to predict the column called **Target**. The input variables are columns labeled as **T1 to T80**. 

## Submission:

Please save and submit this Jupyter notebook file. The correctness of the code matters for your grade. **Readability and organization of your code is also important.** You may lose points for submitting unreadable/undecipherable code. Therefore, use markdown cells to create sections, and use comments where necessary.


# Note:

The data is cleaned up. There are no unqueal length sequences. And, there is no zero padding. So, you shouldn't use any `Masking` layer (like I mentioned in the lecture). 

# Read and Prepare the Data (1 points)

In [1]:
import sklearn.model_selection as model_selection
from sklearn import preprocessing
from sklearn.model_selection import validation_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import itertools
from sklearn import tree
import os
import pandas as pd
from sklearn.utils import resample
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import itertools
from imblearn.over_sampling import SMOTE
from collections import Counter
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Flatten, Dense 
from keras import applications 
from keras.utils.np_utils import to_categorical 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math 
import datetime
import time

In [3]:
df = pd.read_csv("./heartbeat_cleaned.csv")

In [4]:
df.Target.value_counts()

0    4633
4    1584
2    1237
1     445
3      61
Name: Target, dtype: int64

# Find the baseline (0.5 point)

In [5]:
#baseline
df['Target'].value_counts()/len(df)

0    0.582035
4    0.198995
2    0.155402
1    0.055905
3    0.007663
Name: Target, dtype: float64

In [6]:
train = df.drop(columns = ['Target'])
target = df['Target']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train,target, test_size=0.3, random_state=42)

In [8]:

# define oversampling strategy
SMOTE = SMOTE()

# fit and apply the transform
X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)

In [9]:
# summarize class distribution
print("After oversampling: ",Counter(y_train_SMOTE))

After oversampling:  Counter({1: 3217, 0: 3217, 2: 3217, 4: 3217, 3: 3217})


In [10]:
# evaluate the model
def model_eval_results(model, test_x, y_test):
    scores = model.evaluate(test_x, y_test, verbose=0)
    # In results, first is loss, second is accuracy
    # extract the accuracy from model.evaluate

    print("%s: %.2f" % (model.metrics_names[0], scores[0]))
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    return model, scores

### Data Prep for Keras

In [11]:
#Keras
#Keras expects a different input format:
#Data needs to have 3 dimensions
#Convert input variables to a 2-D array with float data type
X_train_SMOTE= np.array(X_train_SMOTE)
X_test= np.array(X_test)
train_x = np.reshape(X_train_SMOTE, (X_train_SMOTE.shape[0], X_train_SMOTE.shape[1], 1))
test_x = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build a cross-sectional shallow model using Keras (with only one hidden layer) (2 points)

### Approach1

In [14]:
# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

visible = keras.layers.Input(shape=(80,))
hidden1 = keras.layers.Dense(65, activation='relu')(visible)
output = keras.layers.Dense(5, activation='softmax')(hidden1)
model_keras_simple = keras.models.Model(inputs=visible, outputs=output)
#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model_keras_simple.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# Fit the model

history = model_keras_simple.fit(train_x, y_train_SMOTE, epochs=50,
                    validation_data=(test_x, y_test))
model_eval_results(model_keras_simple, test_x, y_test)

Epoch 1/50
503/503 [==============================] - 3s 3ms/step - loss: 0.6469 - accuracy: 0.7407 - val_loss: 0.7001 - val_accuracy: 0.7282
Epoch 2/50
503/503 [==============================] - 1s 2ms/step - loss: 0.4233 - accuracy: 0.8293 - val_loss: 0.6121 - val_accuracy: 0.7261
Epoch 3/50
503/503 [==============================] - 1s 2ms/step - loss: 0.3542 - accuracy: 0.8573 - val_loss: 0.4464 - val_accuracy: 0.8534
Epoch 4/50
503/503 [==============================] - 1s 2ms/step - loss: 0.2993 - accuracy: 0.8821 - val_loss: 0.5219 - val_accuracy: 0.8095
Epoch 5/50
503/503 [==============================] - 1s 2ms/step - loss: 0.2629 - accuracy: 0.9003 - val_loss: 0.4103 - val_accuracy: 0.8480
Epoch 6/50
503/503 [==============================] - 2s 5ms/step - loss: 0.2391 - accuracy: 0.9087 - val_loss: 0.3821 - val_accuracy: 0.8572
Epoch 7/50
503/503 [==============================] - 2s 4ms/step - loss: 0.2266 - accuracy: 0.9130 - val_loss: 0.4206 - val_accuracy: 0.8438
Epoch 

(<keras.engine.functional.Functional at 0x7fbe2f618e90>,
 [0.4093019366264343, 0.9053601622581482])

### Approach 2


In [15]:
# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
model_keras_simple = keras.models.Sequential()

model_keras_simple.add(keras.layers.Input(shape=80))
model_keras_simple.add(keras.layers.Dense(65, activation='relu'))
model_keras_simple.add(keras.layers.Dense(5, activation='softmax'))
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model_keras_simple.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# Fit the model

history = model_keras_simple.fit(train_x, y_train_SMOTE, epochs=50,
                    validation_data=(test_x, y_test))
model_eval_results(model_keras_simple, test_x, y_test)

Epoch 1/50
503/503 [==============================] - 2s 2ms/step - loss: 0.6547 - accuracy: 0.7394 - val_loss: 0.7990 - val_accuracy: 0.6487
Epoch 2/50
503/503 [==============================] - 1s 2ms/step - loss: 0.4495 - accuracy: 0.8190 - val_loss: 0.6985 - val_accuracy: 0.6981
Epoch 3/50
503/503 [==============================] - 1s 2ms/step - loss: 0.3502 - accuracy: 0.8658 - val_loss: 0.4078 - val_accuracy: 0.8677
Epoch 4/50
503/503 [==============================] - 1s 2ms/step - loss: 0.3066 - accuracy: 0.8809 - val_loss: 0.3914 - val_accuracy: 0.8576
Epoch 5/50
503/503 [==============================] - 1s 2ms/step - loss: 0.2752 - accuracy: 0.8962 - val_loss: 0.4051 - val_accuracy: 0.8606
Epoch 6/50
503/503 [==============================] - 1s 2ms/step - loss: 0.2518 - accuracy: 0.9056 - val_loss: 0.3393 - val_accuracy: 0.8836
Epoch 7/50
503/503 [==============================] - 1s 2ms/step - loss: 0.2390 - accuracy: 0.9087 - val_loss: 0.3410 - val_accuracy: 0.8827
Epoch 

(<keras.engine.sequential.Sequential at 0x7fbeb9161790>,
 [0.3778935372829437, 0.9133166074752808])

# Build a cross-sectional deep model using Keras (with two or more hidden layers) (2 points)

### Approach 1

In [18]:
# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

visible = keras.layers.Input(shape=(80,))
hidden1 = keras.layers.Dense(65, activation='relu')(visible)
hidden2 = keras.layers.Dense(65, activation='relu')(hidden1)
output = keras.layers.Dense(5, activation='softmax')(hidden2)
model_keras_deep = keras.models.Model(inputs=visible, outputs=output)
#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model_keras_deep.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]
# Fit the model
history = model_keras_deep.fit(train_x, y_train_SMOTE, epochs=50,
                    validation_data=(test_x, y_test), callbacks = callback)
model_eval_results(model_keras_deep, test_x, y_test)

Epoch 1/50
503/503 [==============================] - 3s 4ms/step - loss: 0.6135 - accuracy: 0.7541 - val_loss: 0.6914 - val_accuracy: 0.6993
Epoch 2/50
503/503 [==============================] - 2s 3ms/step - loss: 0.3787 - accuracy: 0.8479 - val_loss: 0.6647 - val_accuracy: 0.7261
Epoch 3/50
503/503 [==============================] - 2s 3ms/step - loss: 0.3055 - accuracy: 0.8841 - val_loss: 0.3527 - val_accuracy: 0.8786
Epoch 4/50
503/503 [==============================] - 2s 3ms/step - loss: 0.2635 - accuracy: 0.8987 - val_loss: 0.3052 - val_accuracy: 0.8961
Epoch 5/50
503/503 [==============================] - 2s 3ms/step - loss: 0.2361 - accuracy: 0.9099 - val_loss: 0.3374 - val_accuracy: 0.8844
Epoch 6/50
503/503 [==============================] - 2s 3ms/step - loss: 0.2170 - accuracy: 0.9185 - val_loss: 0.3969 - val_accuracy: 0.8501
Epoch 7/50
503/503 [==============================] - 2s 3ms/step - loss: 0.2045 - accuracy: 0.9235 - val_loss: 0.2739 - val_accuracy: 0.9062
Epoch 

(<keras.engine.functional.Functional at 0x7fbe2aad4ad0>,
 [0.2901427447795868, 0.9221105575561523])

### Approach 2


In [17]:
#Define the model: for multi-class
# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

#Set the learning rate:
lr=0.001


#Available optimizers:
adagrad = keras.optimizers.Adagrad(learning_rate=lr, epsilon=None, decay=0.0)
sgd = keras.optimizers.SGD(learning_rate=lr, momentum=0.0, decay=0.0, nesterov=False)
rmsprop = keras.optimizers.RMSprop(learning_rate=lr, rho=0.9, epsilon=None, decay=0.0)
adam = keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
nesterov_adam = keras.optimizers.Nadam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

#Initializations:
xavier = keras.initializers.glorot_normal(seed=None)
he = keras.initializers.he_normal(seed=None)


# Activation functions. Uncomment only one
# activation = 'elu' 
activation = 'relu'
#activation = 'tanh'
#activation = 'sigmoid'



#See the droput layers below:
input1 = keras.layers.Input(shape=80)

hidden1 = keras.layers.Dense(70, activation=activation, kernel_initializer=xavier)(input1)
drop1   = keras.layers.Dropout(0.2)(hidden1)
hidden2 = keras.layers.Dense(65, activation=activation, kernel_initializer=xavier)(drop1)
drop2   = keras.layers.Dropout(0.2)(hidden2)
hidden3 = keras.layers.Dense(65, activation=activation, kernel_initializer=xavier)(drop2)

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
output = keras.layers.Dense(5, activation='softmax')(hidden3)

#Compile"
model_keras_deep = keras.Model(inputs = input1, outputs = output)
model_keras_deep.compile(loss='sparse_categorical_crossentropy', 
              optimizer=nesterov_adam, metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]
# Fit the model

history = model_keras_deep.fit(train_x, y_train_SMOTE, 
                    validation_data=(test_x, y_test), 
          epochs=50, callbacks=callback)
model_eval_results(model_keras_deep, test_x, y_test)

Epoch 1/50
503/503 [==============================] - 2s 3ms/step - loss: 0.7845 - accuracy: 0.6934 - val_loss: 0.7119 - val_accuracy: 0.7144
Epoch 2/50
503/503 [==============================] - 1s 2ms/step - loss: 0.4937 - accuracy: 0.8010 - val_loss: 0.5455 - val_accuracy: 0.7986
Epoch 3/50
503/503 [==============================] - 1s 2ms/step - loss: 0.4047 - accuracy: 0.8339 - val_loss: 0.3919 - val_accuracy: 0.8790
Epoch 4/50
503/503 [==============================] - 1s 2ms/step - loss: 0.3574 - accuracy: 0.8589 - val_loss: 0.3352 - val_accuracy: 0.8953
Epoch 5/50
503/503 [==============================] - 1s 2ms/step - loss: 0.3165 - accuracy: 0.8771 - val_loss: 0.4297 - val_accuracy: 0.8400
Epoch 6/50
503/503 [==============================] - 1s 2ms/step - loss: 0.2897 - accuracy: 0.8874 - val_loss: 0.3861 - val_accuracy: 0.8572
Epoch 7/50
503/503 [==============================] - 1s 2ms/step - loss: 0.2680 - accuracy: 0.8987 - val_loss: 0.3370 - val_accuracy: 0.8861
Epoch 

(<keras.engine.functional.Functional at 0x7fbe29068e50>,
 [0.24958212673664093, 0.9233668446540833])

### Data Prep for LSTM and GRU

In [19]:
#Data Prep for LSTM and GRU
X_train_lstm,Y_train_lstm =[],[]
x_train_cv=X_train_SMOTE
y_train=np.array(y_train_SMOTE)
for i in range(1,len(x_train_cv)+1):
    X_train_lstm.append(x_train_cv[i-1:i])
    Y_train_lstm.append(y_train[i-1:i])
X_train_lstm=np.array(X_train_lstm)
Y_train_lstm=np.array(Y_train_lstm)
print(X_train_lstm.shape,Y_train_lstm.shape)
X_test_lstm,Y_test_lstm=[],[]
x_test_cv=X_test
y_test=np.array(y_test)
for i in range(1,len(x_test_cv)+1):
    X_test_lstm.append(x_test_cv[i-1:i])
    Y_test_lstm.append(y_test[i-1:i])
X_test_lstm=np.array(X_test_lstm)
Y_test_lstm=np.array(Y_test_lstm)
print(X_test_lstm.shape,Y_test_lstm.shape) 

(16085, 1, 80) (16085, 1)
(2388, 1, 80) (2388, 1)


# Build a sequential shallow LSTM Model (with only one LSTM layer) (2 points)

In [20]:
# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

model_lstm_simple = keras.models.Sequential()

#model.add(keras.layers.Input(shape=80))
model_lstm_simple.add(keras.layers.LSTM(units=80, return_sequences=True, input_shape=(X_train_lstm.shape[1],X_train_lstm.shape[2])))
model_lstm_simple.add(keras.layers.Dropout(0.2))
model_lstm_simple.add(Dense(5, activation='softmax'))
#model_lstm_simple.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model_lstm_simple.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]
# Fit the model

history = model_lstm_simple.fit(X_train_lstm, Y_train_lstm, epochs=50,
                    validation_data=(X_test_lstm, Y_test_lstm), callbacks=callback)

model_eval_results(model_lstm_simple, X_test_lstm, Y_test_lstm)

Epoch 1/50
503/503 [==============================] - 5s 5ms/step - loss: 0.6510 - accuracy: 0.7363 - val_loss: 0.6963 - val_accuracy: 0.7023
Epoch 2/50
503/503 [==============================] - 2s 4ms/step - loss: 0.4418 - accuracy: 0.8234 - val_loss: 0.5871 - val_accuracy: 0.7404
Epoch 3/50
503/503 [==============================] - 2s 4ms/step - loss: 0.3558 - accuracy: 0.8620 - val_loss: 0.3316 - val_accuracy: 0.8999
Epoch 4/50
503/503 [==============================] - 2s 4ms/step - loss: 0.3172 - accuracy: 0.8744 - val_loss: 0.3893 - val_accuracy: 0.8710
Epoch 5/50
503/503 [==============================] - 2s 3ms/step - loss: 0.2864 - accuracy: 0.8904 - val_loss: 0.3150 - val_accuracy: 0.8932
Epoch 6/50
503/503 [==============================] - 2s 4ms/step - loss: 0.2628 - accuracy: 0.9012 - val_loss: 0.2955 - val_accuracy: 0.9024
Epoch 7/50
503/503 [==============================] - 2s 4ms/step - loss: 0.2475 - accuracy: 0.9089 - val_loss: 0.2624 - val_accuracy: 0.9129
Epoch 

(<keras.engine.sequential.Sequential at 0x7fbe2b232f90>,
 [0.26092979311943054, 0.9183416962623596])

# Build a sequential deep LSTM Model (with only two LSTM layers) (2 points)

In [28]:
# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
model_lstm_deep = keras.models.Sequential()

model_lstm_deep.add(keras.layers.LSTM(units=100,return_sequences=True,input_shape=(X_train_lstm.shape[1],X_train_lstm.shape[2])))
model_lstm_deep.add(keras.layers.Dropout(0.2))
model_lstm_deep.add(keras.layers.LSTM(units=75,return_sequences=True))
model_lstm_deep.add(keras.layers.Dropout(0.1))
model_lstm_deep.add(Dense(5, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model_lstm_deep.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]
# Fit the model

history = model_lstm_deep.fit(X_train_lstm, Y_train_lstm, epochs=50,
                    validation_data=(X_test_lstm, Y_test_lstm), callbacks=callback)

model_eval_results(model_lstm_deep, X_test_lstm, Y_test_lstm)

Epoch 1/50
503/503 [==============================] - 8s 8ms/step - loss: 0.6129 - accuracy: 0.7467 - val_loss: 0.5162 - val_accuracy: 0.8212
Epoch 2/50
503/503 [==============================] - 3s 6ms/step - loss: 0.3767 - accuracy: 0.8479 - val_loss: 0.3863 - val_accuracy: 0.8689
Epoch 3/50
503/503 [==============================] - 3s 5ms/step - loss: 0.3135 - accuracy: 0.8753 - val_loss: 0.3306 - val_accuracy: 0.9062
Epoch 4/50
503/503 [==============================] - 3s 7ms/step - loss: 0.2787 - accuracy: 0.8939 - val_loss: 0.3470 - val_accuracy: 0.8760
Epoch 5/50
503/503 [==============================] - 4s 9ms/step - loss: 0.2579 - accuracy: 0.9023 - val_loss: 0.3152 - val_accuracy: 0.8857
Epoch 6/50
503/503 [==============================] - 4s 8ms/step - loss: 0.2342 - accuracy: 0.9137 - val_loss: 0.3679 - val_accuracy: 0.8714
Epoch 7/50
503/503 [==============================] - 3s 6ms/step - loss: 0.2274 - accuracy: 0.9123 - val_loss: 0.2663 - val_accuracy: 0.9171
Epoch 

(<keras.engine.sequential.Sequential at 0x7fbe25bb8910>,
 [0.2370869219303131, 0.9304857850074768])

# Build a sequential shallow GRU Model (with only one GRU layer) (2 points)

In [22]:
# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
model_gru_simple = keras.models.Sequential()
  
# Defining the cell type
model_gru_simple.add(keras.layers.GRU(80,return_sequences=True, input_shape =(X_train_lstm.shape[1],X_train_lstm.shape[2])))
model_gru_simple.add(keras.layers.Dropout(0.2))
# Defining the densely connected Neural Network layer
# Defining the activation function for the cell
model_gru_simple.add(Dense(5, activation='softmax'))
  
# Defining the optimizing function
optimizer = keras.optimizers.Adam(lr = 0.01)
  
# Configuring the model for training
model_gru_simple.compile(loss ='sparse_categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]
# Fit the model

history = model_gru_simple.fit(X_train_lstm, Y_train_lstm, epochs=50,
                    validation_data=(X_test_lstm, Y_test_lstm), callbacks = callback)

model_eval_results(model_gru_simple, X_test_lstm, Y_test_lstm)

Epoch 1/50
503/503 [==============================] - 4s 4ms/step - loss: 0.6680 - accuracy: 0.7337 - val_loss: 0.7217 - val_accuracy: 0.6872
Epoch 2/50
503/503 [==============================] - 2s 3ms/step - loss: 0.4483 - accuracy: 0.8216 - val_loss: 0.5849 - val_accuracy: 0.7454
Epoch 3/50
503/503 [==============================] - 2s 3ms/step - loss: 0.3580 - accuracy: 0.8596 - val_loss: 0.3396 - val_accuracy: 0.8932
Epoch 4/50
503/503 [==============================] - 2s 3ms/step - loss: 0.3104 - accuracy: 0.8785 - val_loss: 0.4695 - val_accuracy: 0.8095
Epoch 5/50
503/503 [==============================] - 2s 3ms/step - loss: 0.2772 - accuracy: 0.8903 - val_loss: 0.2806 - val_accuracy: 0.9058
Epoch 6/50
503/503 [==============================] - 2s 3ms/step - loss: 0.2549 - accuracy: 0.9031 - val_loss: 0.4175 - val_accuracy: 0.8367
Epoch 7/50
503/503 [==============================] - 2s 3ms/step - loss: 0.2423 - accuracy: 0.9075 - val_loss: 0.3057 - val_accuracy: 0.8928
Epoch 

(<keras.engine.sequential.Sequential at 0x7fbe29e77090>,
 [0.3337429463863373, 0.8835846185684204])

# Build a sequential deep GRU Model (with only two GRU layers) (2 points)

In [29]:
# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
model_gru_deep = keras.models.Sequential()
  
# Defining the cell type
model_gru_deep.add(keras.layers.GRU(80,return_sequences=True, input_shape =(X_train_lstm.shape[1],X_train_lstm.shape[2])))
model_gru_deep.add(keras.layers.Dropout(0.2))
# Defining the densely connected Neural Network layer
model_gru_deep.add(keras.layers.GRU(units=75,return_sequences=True))
model_gru_deep.add(keras.layers.Dropout(0.1))
# Defining the activation function for the cell
model_gru_deep.add(Dense(5, activation='softmax'))
  
# Defining the optimizing function
optimizer = keras.optimizers.Adam(lr = 0.01)
  
# Configuring the model for training
model_gru_deep.compile(loss ='sparse_categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]
# Fit the model

history = model_gru_deep.fit(X_train_lstm, Y_train_lstm, epochs=50,
                    validation_data=(X_test_lstm, Y_test_lstm), callbacks = callback)

model_eval_results(model_gru_deep, X_test_lstm, Y_test_lstm)

Epoch 1/50
503/503 [==============================] - 8s 8ms/step - loss: 0.6149 - accuracy: 0.7491 - val_loss: 0.5833 - val_accuracy: 0.7772
Epoch 2/50
503/503 [==============================] - 3s 6ms/step - loss: 0.3735 - accuracy: 0.8480 - val_loss: 0.3890 - val_accuracy: 0.8543
Epoch 3/50
503/503 [==============================] - 3s 6ms/step - loss: 0.3037 - accuracy: 0.8818 - val_loss: 0.3092 - val_accuracy: 0.8995
Epoch 4/50
503/503 [==============================] - 3s 6ms/step - loss: 0.2780 - accuracy: 0.8923 - val_loss: 0.3491 - val_accuracy: 0.8744
Epoch 5/50
503/503 [==============================] - 3s 6ms/step - loss: 0.2529 - accuracy: 0.9027 - val_loss: 0.2947 - val_accuracy: 0.8894
Epoch 6/50
503/503 [==============================] - 3s 6ms/step - loss: 0.2316 - accuracy: 0.9144 - val_loss: 0.3083 - val_accuracy: 0.8890
Epoch 7/50
503/503 [==============================] - 3s 6ms/step - loss: 0.2238 - accuracy: 0.9171 - val_loss: 0.2743 - val_accuracy: 0.9062
Epoch 

(<keras.engine.sequential.Sequential at 0x7fbe25b0ad50>,
 [0.26514485478401184, 0.9191792011260986])

# Discussion

## List the test values of each model you built (0.5 points)

In [30]:
predictions_keras_simple = model_keras_simple.predict(X_test)
predictions_keras_simple = np.argmax(predictions_keras_simple, axis=1)
predictions_keras_deep = model_keras_deep.predict(X_test)
predictions_keras_deep = np.argmax(predictions_keras_deep, axis=1)
predictions_lstm_simple = model_lstm_simple.predict(X_test_lstm)
predictions_lstm_simple = list(itertools.chain(*np.argmax(predictions_lstm_simple, axis=2)))
predictions_lstm_deep = model_lstm_deep.predict(X_test_lstm)
predictions_lstm_deep = list(itertools.chain(*np.argmax(predictions_lstm_deep, axis=2)))
predictions_gru_simple = model_gru_simple.predict(X_test_lstm)
predictions_gru_simple = list(itertools.chain(*np.argmax(predictions_gru_simple, axis=2)))
predictions_gru_deep = model_gru_deep.predict(X_test_lstm)
predictions_gru_deep = list(itertools.chain(*np.argmax(predictions_gru_deep, axis=2)))

In [31]:
class_map = {0 : 'Normal',
            1 : 'Supraventricular premature beat',
            2 : 'Premature ventricular contraction',
            3 : 'Fusion of ventricular and normal beat',
            4 : 'Unclassifiable beat'}

### listed test values for each model

In [32]:
print(f'keras simple preds:{list(map(lambda x: class_map[x],predictions_keras_simple))}\n'\
      f'keras deep preds:{list(map(lambda x: class_map[x],predictions_keras_deep))}\n'\
      f'lstm simple preds:{list(map(lambda x: class_map[x],predictions_lstm_simple))}\n'\
      f'lstm deep preds:{list(map(lambda x: class_map[x],predictions_lstm_deep))}\n'\
      f'gru simple preds:{list(map(lambda x: class_map[x],predictions_gru_simple))}\n'\
      f'gru deep preds:{list(map(lambda x: class_map[x],predictions_gru_deep))}\n'\
      )

keras simple preds:['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Unclassifiable beat', 'Unclassifiable beat', 'Premature ventricular contraction', 'Unclassifiable beat', 'Normal', 'Normal', 'Normal', 'Normal', 'Supraventricular premature beat', 'Premature ventricular contraction', 'Normal', 'Premature ventricular contraction', 'Normal', 'Normal', 'Unclassifiable beat', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Premature ventricular contraction', 'Normal', 'Premature ventricular contraction', 'Unclassifiable beat', 'Normal', 'Premature ventricular contraction', 'Supraventricular premature beat', 'Normal', 'Unclassifiable beat', 'Supraventricular premature beat', 'Normal', 'Premature ventricular contraction', 'Unclassifiable beat', 'Normal', 'Normal', 'Normal', 'Unclassifiable beat', 'Supraventricular premature beat', 'Normal', 'Supraventricular premature beat', 'Premature ventricular contraction', 'Normal', 'Normal', 'Normal', 'Normal', 'Norm

## Which model performs the best and why? (0.5 points) 
## How does it compare to baseline? (0.5 points)

In [ ]:
"""keras, GRU, LSTM with 2 layers are better for simpler data like in our case, and they performed better\
  compared to simpler(1 layer) networks
  If the dataset is small, then GRU is preferred otherwise LSTM for the larger dataset.
  GRU exposes the complete memory and hidden layers but LSTM doesn't.
  also,  GRU is 29.29% faster than LSTM for processing the same dataset
  hence as the accuracies are almost similar: GRU/simpler keras is better(faster) because the data we trained is simpler
  and ""